In [1]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
import torchaudio

In [2]:
root_dir = "/hdd/sound_sn/SoccerNet-code/data/"
train_file = "/hdd/SoccerNet-code/data/listgame_Train_300.npy"
valid_file = "/hdd/SoccerNet-code/data/listgame_Valid_100.npy"
test_file = "/hdd/SoccerNet-code/data/listgame_Test_100.npy"

# Librosa test

In [3]:
import librosa
import matplotlib.pyplot as plt

In [4]:
SAMPLE_RATE = 8000 # Should be 16 kHz
WINDOW_LENGTH = int(0.025 * SAMPLE_RATE) # Should be 25 ms
N_FFT = 512 # Should be 512
STEP_SIZE = int(0.01 * SAMPLE_RATE) # Should be 10ms


In [5]:
# Clip dataset
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch
import os
import json
import torchvision
import datetime
import torchaudio
import librosa
from subprocess import Popen, PIPE
import re



class SoccerNetDataset(Dataset):
    """Soccernet Dataset"""
    
    SAMPLE_RATE = 8000 # Should be 16 kHz
    WINDOW_LENGTH = int(0.025 * SAMPLE_RATE) # Should be 25 ms
    N_FFT = 512 # Should be 512
    STEP_SIZE = int(0.01 * SAMPLE_RATE) # Should be 10ms
    
    def __init__(self,npy_file,
                 root_dir,
                 transform=None,
                 background=False,
                 wsize=4):
    
        self.wsize = wsize
        self.npy_file = np.load(npy_file)
        self.samples = list() # maybe change structure later depending on efficiency        
        self.root_dir = root_dir
        self.transform = transform

        for e in self.npy_file:
            path, annotations = self.get_annotations(e)

            duration1 = self.getVideoLength(self.root_dir + e + "/1.mkv")
            duration2 = self.getVideoLength(self.root_dir + e + "/2.mkv")
            #print(f"duration1 : {duration1}, duration2: {duration2}")
            for annotation in annotations:
                # Check that annotations hold correct labels
                        if ("card" in annotation["label"]) or ("subs" in annotation["label"]) or ("soccer" in annotation["label"]):
                            annotation["duration1"] = duration1
                            annotation["duration2"] = duration2
                            self.samples.append([path,annotation])

       
    def __len__(self):
        return len(self.samples)
    
    def getVideoLength(self,video_file):
        res = Popen(['ffmpeg', '-i', video_file, '-hide_banner'],stdout=PIPE,stderr=PIPE)
        none,meta = res.communicate()
        meta_out = meta.decode()
        #---| Take out info
        duration = re.search(r'Duration:.*', meta_out)
        return duration.group()[:21]

    def __getitem__(self,idx):
        """Returns a sample containing video path, clip and label"""
        if torch.is_tensor(idx):
            idx.tolist()
        
        # get annotations
        time_half = int(self.samples[idx][1]["gameTime"][0])
        time_minute = int(self.samples[idx][1]["gameTime"][-5:-3])
        time_second = int(self.samples[idx][1]["gameTime"][-2:])
        annotation = self.samples[idx][1]

        # Get label
        if ("card" in annotation["label"]): label = 0
        elif ("subs" in annotation["label"]): label = 1
        elif ("soccer" in annotation["label"]): label = 2
        elif ("background" in annotation["label"]): label = 3
        else: 
            print("Warning, label not compatible with set")
            return

        # Get audiopath
        audiopath = os.path.join(self.root_dir,
                              str(self.samples[idx][0]),
                                str(time_half)+"_audio.wav")
        

        
        one_hot_label = np.zeros(4)
        one_hot_label[label] = 1
        # Get video frames 
        
        # get start in second, use labeled time as center TODO: fix centerframe as keyframe and stride
        fps = 25.0 # assume fps = 25 for now, should be so
        start_sec = time_minute*60 + time_second
        end_sec = start_sec
        
        if start_sec == 0:
            end_sec += (1/fps) # possibly unstable solution
            
        end_sec = end_sec + self.wsize # might need to subtract 1/fps
        # Shift backwards to center around time but check that time > 0
        diff = (end_sec - start_sec) / 2 # TODO : Might result in bad precision
        temp_start_sec = start_sec - diff
        temp_end_sec = end_sec - diff

        # Only change as long as the shift operation doesnt shift out of bounds 
        if temp_start_sec >= 0:
            start_sec = temp_start_sec
            end_sec = temp_end_sec
        
        # Buffer to endsec incase of bad load
        end_sec = end_sec + 0.9 # loads more frames than needed, then reduced later
        
        y, sr = librosa.load(audiopath,sr=SAMPLE_RATE,offset=start_sec,duration=self.wsize+1)
        
        print(len(y))
        y = y[:SAMPLE_RATE*4]
        
        ms = torchaudio.transforms.MelSpectrogram(sample_rate=SAMPLE_RATE,
                                                n_fft=N_FFT,
                                                win_length=WINDOW_LENGTH,
                                                hop_length=STEP_SIZE)(torch.Tensor(y)).log10().unsqueeze(0)
        print(f"ms size = {ms.size()}")
        
        ms = ms[:,:,:401]
        if ms.size() != (1,128,401):
            print(f"ms size :{ms.size()}, using zeros instead ...")
            ms = torch.zeros((1,128,401))
        
        sample = {'audiopath': audiopath, 'annotation':annotation,'idx':idx,'audio':y,'sr':sr,'mel_spectogram':ms, 'one_hot_label':one_hot_label,'label':label}
        
        return sample
            
    def get_annotations(self,path):
        """ Reads json files and returns """
        with open(self.root_dir+path+"/Labels.json") as jsonfile:
            json_label = json.load(jsonfile)
        
        labels = [e for e in json_label['annotations']]
        
        return path,labels
    def get_keyframe(self,idx):
        if self.frame_center == 'back': return self.__getitem__(idx)['clip'][0,:,:,:]
        elif self.frame_center == 'center': return self.__getitem__(idx)['clip'][self.nframes//2,:,:,:]
        elif self.frame_center == 'front': return self.__getitem__(idx)['clip'][self.nframes-1,:,:,:]
    def describe(self):
        card = 0
        subs = 0
        goal = 0
        background = 0

        for sample in self.samples:
            annotation = sample[1]
        # Get label
            if ("card" in annotation["label"]): card += 1
            elif ("subs" in annotation["label"]): subs +=1
            elif ("soccer" in annotation["label"]): goal += 1
            elif ("background" in annotation["label"]): background += 1

        print("Description of dataset\n\n")
        print("\n ********* Classes *********")
        print("\n card = 0\n subs = 1\n goals = 2\n background = 3")

        print("\n ********* Distribution and count *********")
        print(f"\n N card: {card} \n N subs: {subs} \n N goal: {goal} \n N background: {background} \n \n Total : {card+subs+goal+background}")
        
        print("\n\n ********* Configuration *********")
        print(f"\n npy_file: {self.npy_file} \n tshift: {self.tshift} \n root_dir: {self.root_dir} \n transform: {self.transform} \n frame_center: {self.frame_center} \n nframes: {self.nframes} \n stride_frames: {self.stride_frames} \n background: {self.background}")
        print("\n\n ********* End of description *********")

In [6]:
root_dir = "/hdd/sound_sn/SoccerNet-code/data/"
train_file = "/hdd/SoccerNet-code/data/listgame_Train_300.npy"

In [7]:
X = SoccerNetDataset(root_dir=root_dir,npy_file=train_file)

In [8]:
params = {'batch_size': 4,
         'shuffle': False,
         'num_workers':4}

In [9]:
dataloader = DataLoader(X,**params)

In [10]:
%%bash
nvidia-smi

Wed Oct 14 14:07:02 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 450.66       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   47C    P8    30W / 260W |    826MiB / 11018MiB |     11%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Simple network

In [11]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.bn1 = nn.BatchNorm2d(6)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.bn2 = nn.BatchNorm2d(16)
        self.pool = nn.MaxPool2d(4, 4)
        self.fc1 = nn.Linear(2208, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 2208)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [12]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

In [13]:
# Disable warnings
import warnings
warnings.filterwarnings('ignore')

# Note: Can also see warning once
# warnings.filterwarnings(action='once')

In [14]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/mel_spec_experiment_2')

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net.to(device)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=2208, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=4, bias=True)
)

In [16]:
net.to(device)
# Dataloader or dataset faulty(returns batches of 0
epochs = 2
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        print(data['label'])
        inputs, labels = data['mel_spectogram'].to(device),data['label'].to(device)

        #print(inputs.size())
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 5 == 0:    # print every 2000 mini-batches
            
            writer.add_scalar('training loss',
                            running_loss / 5,
                            epoch * len(dataloader) + i)
            
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 5))
            running_loss = 0.0

print('Finished Training')

40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
4000040000

ms size = torch.Size([1, 128, 401])
ms size = torch.Size([1, 128, 401])
tensor([2, 0, 0, 1])
[1,     1] loss: 0.280
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
tensor([0, 0, 1, 1])
tensor([2, 1, 2, 0])
tensor([0, 2, 0, 1])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms si

ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
tensor([0, 1, 1, 1])
tensor([1, 1, 0, 2])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
tensor([2, 2, 2, 0])
tensor([2, 1, 1, 0])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
tensor([1, 1, 1, 2])
[1,    46] loss: 1.227
40000
ms size = torch.Size([1, 128, 40

40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
tensor([0, 1, 1, 0])
tensor([0, 0, 1, 2])
tensor([2, 1, 0, 1])
tensor([1, 2, 1, 1])
[1,    86] loss: 1.133
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms si

ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
tensor([0, 1, 1, 0])
tensor([0, 2, 2, 0])
tensor([0, 0, 0, 1])
tensor([1, 1, 1, 1])
[1,   126] loss: 1.075
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
tensor([0, 2, 2, 1])
tensor([2, 0, 2, 2])
tensor([1, 1, 1, 2])
tensor([0, 1, 1, 0])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = t

ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
tensor([1, 2, 1, 1])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
tensor([1, 1, 0, 0])
tensor([1, 1, 1, 1])
[1,   171] loss: 0.888
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
tensor([0, 0, 2, 1])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = t

...

ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
tensor([1, 1, 2, 1])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
tensor([0, 0, 0, 0])
tensor([0, 2, 0, 0])
tensor([1, 1, 0, 1])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401]

ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
tensor([2, 2, 0, 0])
tensor([1, 1, 2, 1])
tensor([1, 2, 1, 1])
tensor([0, 0, 0, 0])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
tensor([0, 0, 0, 0])
[2,   831] loss: nan
tensor([1, 1, 1, 0])
tensor([1, 2, 1, 1])
tensor([2, 2, 1, 2])
40000
ms size = tor

ms size = torch.Size([1, 128, 401])
tensor([1, 0, 1, 0])
[2,   871] loss: nan
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
tensor([0, 0, 0, 2])
tensor([0, 0, 0, 2])
tensor([1, 0, 1, 1])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
tensor([1, 2, 1, 1])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401]

ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
tensor([1, 1, 1, 0])
tensor([1, 2, 0, 0])
tensor([0, 2, 0, 1])
tensor([2, 1, 1, 0])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401]

ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
tensor([1, 1, 1, 0])
[2,   956] loss: nan
tensor([0, 2, 2, 0])
tensor([2, 1, 2, 0])
tensor([1, 1, 0, 1])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = torch.Size([1, 128, 401])
40000
ms size = tor

In [ ]:
net.conv2.weight.sum()

In [ ]:
writer.close()

# Evaluate samples to look for cases where y, sr from librosa is inconsistent

In [ ]:
outputs = net(inputs)

In [ ]:
inputs.size()

In [ ]:
net.fc1.weight

# With GPU

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
net.to(device)

running_loss = 0.0
for i, data in enumerate(dataloader, 0):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data['mel_spectogram'].unsqueeze(0).to(device),torch.argmax(data['one_hot_label']).to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    if i % 2000 == 1999:    # print every 2000 mini-batches
        print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss / 2000))
        running_loss = 0.0

print('Finished Training')

In [ ]:
%%bash
nvidia-smi